<h1>Intro to Computer Music, Lab07</h1>
<h2>Gus Xia, NYU Shanghai</h2>

In this lab you will:

1. learn how to use pretty_midi library to generate MIDI in python
2. We focus on the basics this week: Note-on and Note-off
2. Use MIDI to recreate a simple composition by coding

<h2> Load packages </h2>


In [ ]:
# To begin using librosa we need to import it, and other tools such as matplotlib and numpy
from pylab import *
import pretty_midi
import librosa             # The librosa library
import librosa.display     # librosa's display module (for plotting features)
import IPython.display     # IPython's display module (for in-line audio)
import matplotlib.pyplot as plt # matplotlib plotting functions
import matplotlib.style as ms   # plotting style
import numpy as np              # numpy numerical functions
ms.use('seaborn-muted')         # fancy plot designs
from __future__ import print_function # use the print() function from Python3



<h2>Pretty MIDI Library</h2>

Let's first install pretty_midi: 

a) Download the package from https://github.com/craffel/pretty-midi

b) In the terminal, go to the folder, and run "python setup.py install"

On my Mac, step b) looks like:

Then, use pretty MIDI library to create a simple chord:

In [ ]:
import pretty_midi
# Create a PrettyMIDI object
cello_music = pretty_midi.PrettyMIDI()
# Create an Instrument instance for a cello instrument
cello_program = pretty_midi.instrument_name_to_program('Cello')
cello = pretty_midi.Instrument(program=cello_program)
# Iterate over note names, which will be converted to note number later
for note_name in ['C4', 'E4', 'G4','C5', 'E5', 'G5','C6']:
    # Retrieve the MIDI note number for this note name
    note_number = pretty_midi.note_name_to_number(note_name)
    # Create a Note instance for this note, starting at 0s and ending at 3s
    note = pretty_midi.Note(velocity=100, pitch=note_number, start=0, end=3)
    # Add it to our cello instrument
    cello.notes.append(note)
# Add the cello instrument to the PrettyMIDI object
cello_music.instruments.append(cello)
# Write out the MIDI data
cello_music.write('cello-C-chord.mid')


run the code above, now you should be able to find the midi file "cello-C-chord.mid" in the homework folder. If you want to use another instrument, simply substitute "Cello" with any standard MIDI instrument. A list can be found here http://www.pgmusic.com/tutorial_gm.htm

<h2>More of Data structure:</h2>


In [ ]:
print(cello_music)
print(cello_music.instruments)
print(cello_music.instruments[0])
print(cello)
print(cello_music.instruments[0] == cello)

Let's look at the notes inside the cello. It is a list, where each element is a "Note" object.

In [ ]:
cello.notes

<h2>A More Efficient Way for Music Generation:</h2>


We can also see this note list as a 7 by 4 matrix, and create/manipulate the matrix by built-in numpy methods. 

In [ ]:
#import numpy library
from pylab import *
%matplotlib inline 

# create a 3 * 4 matrix, with initial values 0
note_matrix = matrix(zeros((7,4)))
print ("initial matrix is:\n", note_matrix)
### fill the matrix. 
# For a matrix M, numpy uses M[row_index, colume_index] to access the element
# M[:, column_index] returns an entire colume, and the same principle applies to 
# M[row_index, :]
note_matrix[:,2] = matrix([60,64,67,72,76,79,84]).T #.T is transpose
# start, end, and velocity. This assignment can be done in one line
# Note that the initial index is 0
note_matrix[:,[0,1,3]] = [0,3,100]
print ("now the matrix is:\n", note_matrix)

Two handy functions: We can then call the following function to transfer the matrix back to "pretty_midi note" and display the piano roll.

In [ ]:
# The next line makes all plots appear in the notebook, instead of a separate pop-up window
%matplotlib inline
# visualize score
def show_score(S, fs = 100):
    imshow(S, aspect='auto', origin='bottom', interpolation='nearest', cmap=cm.gray_r)
    xlabel('Time')
    ylabel('Pitch')
    pc=array(['C','C#','D','Eb','E','F','F#','G','Ab','A','Bb','B'])
    idx = tile([0,4,7],13)[:128]
    yticks(arange(0,128,4),pc[idx], fontsize=5)
    xticks(arange(0,S.shape[1],fs),arange(0,S.shape[1],fs)/fs, )     
def matrix2notes(m):
    # first sort the matrix to make sure starting time is in order
    m = array(m)
    m = m[argsort(m[:,0]),:]
    # transfer the databack to note list
    notes=[pretty_midi.Note(start=m[i,0], end=m[i,1], pitch=int(m[i,2]), velocity= int(m[i,3]) ) 
                            for i in range(size(m,0))]
    return notes


In [ ]:
length = size(note_matrix,0)
# change starting time, ending time, and velocity by one line command
note_matrix[:,[0,1,3]] = matrix([[x for x in range(length)],
                                 [x+1 for x in range(length)],
                                 [100- x**2.5 for x in range(length)]]).T
print ("we've got an arpeggio with decreasing velocity:\n", note_matrix)
cello.notes = matrix2notes(note_matrix)
# show_score(cello_music.get_piano_roll(fs = 10), fs = 10)
show_score(cello_music.get_piano_roll())
wave = cello.synthesize(fs = 44100)
IPython.display.Audio(data=wave, rate=44100) # press the "play" button to hear audio


<h4>Pitch bends</h4>


Since MIDI notes are all defined to have a specific integer pitch value, in order to represent arbitrary pitch frequencies we need to use pitch bends. A PitchBend class in pretty_midi holds a time (in seconds) and a pitch offset. The pitch offset is an integer in the range [-8192, 8191], which in General MIDI spans the range from -2 to +2 semitones. As with Notes, the Instrument class has a list for PitchBend class instances.

In [ ]:
# try to add only one pitch bend msg
print(cello.pitch_bends)
cello.pitch_bends.append(pretty_midi.PitchBend(8192/2, 0.5))

adding lots of pitch bend in a row

In [ ]:
# how about adding a lot
for t in arange(0,7,0.01):
    cello.pitch_bends.append(pretty_midi.PitchBend(int(8192*(t % 1)), t))

show_score(cello_music.get_piano_roll())
wave = cello.synthesize(fs = 44100)
IPython.display.Audio(data=wave, rate=44100) # press the "play" button to hear audio


<h4>fractal music</h4>


In [ ]:
# a simple fractal music funtion which returns a note matrix 
Trans = 7
Vel = 100
def fractal_music(start, end, layers, base):
    # print (start, end, layers, base)
    if layers > 1:
        return concatenate((fractal_music(start,(start + end)/2.,layers-1,base + Trans),
                            fractal_music((start+end)/2.,end,layers-1,base + Trans + 3),
                            fractal_music(start,end,1,base)),0)
    else:
        return matrix([start, end, base, Vel])

# create an 8 secs fractal music with 5 layers. The base note is 48.
note_matrix = fractal_music(0,8,9,40)
cello.notes = matrix2notes(note_matrix)
cello.pitch_bends = []
# write the file
cello_music.write('simple-fractal.mid')
show_score(cello_music.get_piano_roll())
wave = cello.synthesize(fs = 44100)
IPython.display.Audio(data=wave, rate=44100) # press the "play" button to hear audio


<h2> To do:</h2>

1. Try to create your own fractal music
2. Try to add pitch bend into your music in a creative way
3. Optional: Try some other "deterministic algorithms" (page 14&15) covered in class
3. To be continued in Lab07-extend


In [ ]:
# 杨辉三角
# Kinda fractal, kinda deterministic. 

yhMusic = pretty_midi.PrettyMIDI()
cello_program = pretty_midi.instrument_name_to_program('music box')
cello = pretty_midi.Instrument(program=cello_program)
yhMusic.instruments.append(cello)
drum = pretty_midi.Instrument(program=115)
yhMusic.instruments.append(drum)

MOD = 21
MEASURES = 136
SLOW = 1.7

def num2pitch(num):
    step = num % 7
    height = num // 7
    step_ = {0:0,
            1:2,
            2:4,
            3:6,
            4:7,
            5:9,
            6:11}[step]
    return 55 - 2 + height * 12 + step_

def num2pitch(num):
    step = num % 5
    height = num // 5
    step_ = {0:0,
            1:2,
            2:4,
            3:7,
            4:9}[step]
    return 55 - 2 + height * 12 + step_

def num2pitch(num):
    step = num % 12
    height = num // 12
    return 55 - 2 + height * 12 + step

def num2pitch(num):
    step = num % 8
    height = num // 8
    step_ = {0:0,
            1:2,
            2:4,
            3:5,
            4:6,
            5:7,
            6:9,
            7:11}[step]
    return 55 - 2 + height * 12 + step_

def num2pitch(num):
    step = num % 4
    height = num // 4
    step_ = {0:5,
            1:7,
            2:11,
            3:14}[step]
    return 48 + height * 12 + step_

def doTake(measure):
    return True
    if measure <= 17:
        return True
    if measure in range(24, 38):
        return True
    if measure in range(46, 64):
        return True
    if measure in range(77, 81):
        return True
    if measure in range(96, 111):
        return True
    return False

last_line = [1]
last_len = 1
notes=[]
real_m = 0
for measure in range(MEASURES):
    line = [1]
    for i in range(last_len - 1):
        line.append((last_line[i]+last_line[i+1]) % MOD)
    line.append(1)
    now_len = last_len + 1
    if doTake(measure):
        real_m += 1
        last_pitch = None
        for i, num in enumerate(line[:-1]):
            if i == 0:
                vel = 100;
            else:
                vel = 60;
            pit = num2pitch(num)
            notes.append([vel, pit, SLOW*(real_m + i/(now_len-1)), 
                                    SLOW*(real_m + (i+1)/(now_len-1))])
            if last_pitch is not None:
                if last_pitch == pit:
                    notes.pop(-1)
                    notes[-1][3] = SLOW*(real_m + (i+1)/(now_len-1))
            last_pitch = pit
    last_line = line
    last_len = now_len
for note in notes:
    cello.notes.append(pretty_midi.Note(*note))

for i in range(real_m):
    drum.notes.append(pretty_midi.Note(100, 30, SLOW * i, SLOW * (i+1)))
    drum.notes.append(pretty_midi.Note(60, 40, SLOW * (i+.5), SLOW * (i+1)))

yhMusic.write('yh.mid')
# wave = cello.synthesize(fs = 44100)
# IPython.display.Audio(data=wave, rate=22050) # press the "play" button to hear audio